# Your first scraper
In this project, we will guide you step by step through the process of:

1. creating a self-contained development environment.
1. retrieving some information from an API (a website for computers)
2. leveraging it to scrape a website that does not provide an API
3. saving the output for later processing

Here we query an API for a list of countries and their past leaders. We then extract and sanitize their short bio from Wikipedia. Finally, we save the data to disk.

This task is often the first (coding) step of a datascience project and you will often come back to it in the future.

You will study topics such as *scraping*, *data structures*, *regular expressions*, *concurrency* and *file handling*. We will point out useful resources at the appropriate time. 

Let's dive in!

## Creating a clean environment

Use the [`venv`](https://docs.python.org/3/library/venv.html) command to create a new environment called `wikipedia_scraper_env`.

Activate it.

You will find more info about virtual environments in the course content and on the web.

## API Scraping

### A simple API query
You will start with the basics: how to do a simple request to an [API endpoint](../../2.python/2.python_advanced/05.Scraping/5.apis.ipynb).

You will use the [requests](https://requests.readthedocs.io/en/latest/) external library through the `import` keyword.

NOTE: external libraries need to be installed first. Check their documentation.

Check the [Quickstart](https://requests.readthedocs.io/en/latest/user/quickstart/) section of the Requests documentation to:
1. use the `get()` method to connect to this endpoint: https://country-leaders.onrender.com/status
2. check if the `status_code` is equal to 200, which means OK.
    * if OK, `print()` the `text`` of the response.
    * if not, `print()` the `status_code`. 

Here is the signification of [HTTP status codes](https://en.wikipedia.org/wiki/List_of_HTTP_status_codes).


In [20]:
# import the requests library (1 line)
import requests 
# assign the root url (without /status) to the root_url variable for ease of reference (1 line)
root_url = "https://country-leaders.onrender.com"
# assign the /status endpoint to another variable called status_url (1 line)
status_url = "/status"
# query the /status endpoint using the get() method and store it in the req variable (1 line)
req = requests.get(root_url + status_url)
# check the status_code using a condition and print appropriate messages (4 lines)
if req.status_code == 200:
    print("It's good")
else:
    print("There was an error")

It's good


### Dealing with JSON

[JSON](https://quickref.me/json) is the preferred format to deal with data over the web. You cannot avoid it so you would better get acquainted.

Connect to another endpoint called `/countries` but this time the API will return data in the JSON format. Check what to do in the Requests [Quickstart](https://requests.readthedocs.io/en/latest/user/quickstart/).


In [21]:
# Set the countries_url variable (1 line)
countries_url = "https://country-leaders.onrender.com/countries"
# query the /countries endpoint using the get() method and store it in the req variable (1 line)
req = requests.get(countries_url)
# Get the JSON content and store it in the countries variable (1 line)
countries = req.json()
# display the request's status code and the countries variable (1 line)
print(req.status_code, countries)


403 {'message': 'The cookie is missing'}


### Cookies anyone?

It looks like the access to this API is restricted...
Query the `/cookie` endpoint and extract the appropriate field to access your cookie.

You will need to use this cookie in each of the following API requests.

In [22]:
# Set the cookie_url variable (1 line)
cookie_url = "https://country-leaders.onrender.com/cookie"
# Query the endpoint, set the cookies variable and display it (2 lines)
req = requests.get(cookie_url)
cookies = req.cookies
print(cookies)



<RequestsCookieJar[<Cookie user_cookie=e884f154-1d9e-4fb5-a0b2-33b3b89b3c7e for country-leaders.onrender.com/>]>


Try to query the countries endpoint using the cookie, save the output and print it.

Chances are the cookie has expired... Thanksfully, you got a nice error message. For now, simply execute the last 2 cells quickly so you get a result.

In [23]:
# query the /countries endpoint, assign the output to the countries variable (1 line)
countries = requests.get("https://country-leaders.onrender.com/countries", cookies=cookies)
# display the countries variable (1 line)
print(countries)


<Response [200]>


### Getting the actual data from the API

Query the `/leaders` endpoint.

In [24]:
# Set the leaders_url variable (1 line)
leaders_url = "https://country-leaders.onrender.com/leaders"
# query the /leaders endpoint, assign the output to the leaders variable (1 line)
leaders = requests.get("https://country-leaders.onrender.com/leaders")
# display the leaders variable (1 line)
print(leaders)

<Response [403]>


It looks like this endpoint requires additional information in order to return its result. Check the API [*documentation*](https://country-leaders.onrender.com/docs) in your web browser.

Change the query to accept *parameters*. You should know where to find help by now.

In [25]:
# query the /leaders endpoint using cookies and parameters (take any country in countries)
# assign the output to the leaders variable (1 line)
leaders= requests.get("https://country-leaders.onrender.com/leaders",params={"country": "be"}, cookies=cookies).json()
# display the leaders variable (1 line)
print(leaders)


[{'id': 'Q12978', 'first_name': 'Guy', 'last_name': 'Verhofstadt', 'birth_date': '1953-04-11', 'death_date': None, 'place_of_birth': 'Dendermonde', 'wikipedia_url': 'https://nl.wikipedia.org/wiki/Guy_Verhofstadt', 'start_mandate': '1999-07-12', 'end_mandate': '2008-03-20'}, {'id': 'Q12981', 'first_name': 'Yves', 'last_name': 'Leterme', 'birth_date': '1960-10-06', 'death_date': None, 'place_of_birth': 'Wervik', 'wikipedia_url': 'https://nl.wikipedia.org/wiki/Yves_Leterme', 'start_mandate': '2009-11-25', 'end_mandate': '2011-12-06'}, {'id': 'Q12983', 'first_name': 'Herman', 'last_name': 'None', 'birth_date': '1947-10-31', 'death_date': None, 'place_of_birth': 'Etterbeek', 'wikipedia_url': 'https://nl.wikipedia.org/wiki/Herman_Van_Rompuy', 'start_mandate': '2008-12-30', 'end_mandate': '2009-11-25'}, {'id': 'Q14989', 'first_name': 'Léon', 'last_name': 'Delacroix', 'birth_date': '1867-12-27', 'death_date': '1929-10-15', 'place_of_birth': 'Saint-Josse-ten-Noode', 'wikipedia_url': 'https://nl

### A sneak peak at the data (finally)

Look inside a few examples. Notice the dictionary keys available for each entry. You have your first example of *structured data*. This data was sanitized for your benefit, meaning it is readily exploitable without modification.

You will also notice there is a Wikipedia link for each entry. You will need to extract additional information there. This will be a case of *semi-structured* data.

The /countries endpoint returns a `list` of several country codes.

You need to loop through this list and query the /leaders endpoint for each one. Save each `json` result in a dictionary called `leaders_per_country`.

In [26]:
# 4 lines
countries= requests.get("https://country-leaders.onrender.com/countries", cookies=cookies).json()
leaders_per_country = {}
for country in countries:
    leaders= requests.get("https://country-leaders.onrender.com/leaders", params={"country": country},cookies=cookies).json()
    leaders_per_country[country] = leaders




In [27]:
# or 1 line
print(leaders_per_country)

{'ma': [{'id': 'Q57553', 'first_name': 'Mohammed', 'last_name': 'None', 'birth_date': '1963-08-21', 'death_date': None, 'place_of_birth': 'Rabat', 'wikipedia_url': 'https://ar.wikipedia.org/wiki/%D9%85%D8%AD%D9%85%D8%AF_%D8%A7%D9%84%D8%B3%D8%A7%D8%AF%D8%B3_%D8%A8%D9%86_%D8%A7%D9%84%D8%AD%D8%B3%D9%86', 'start_mandate': '1999-07-23', 'end_mandate': None}, {'id': 'Q69103', 'first_name': 'Hassan', 'last_name': 'None', 'birth_date': '1929-07-09', 'death_date': '1999-07-20', 'place_of_birth': 'Rabat', 'wikipedia_url': 'https://ar.wikipedia.org/wiki/%D8%A7%D9%84%D8%AD%D8%B3%D9%86_%D8%A7%D9%84%D8%AB%D8%A7%D9%86%D9%8A_%D8%A8%D9%86_%D9%85%D8%AD%D9%85%D8%AF', 'start_mandate': '1961-02-26', 'end_mandate': '1999-07-23'}, {'id': 'Q193874', 'first_name': 'Mohammed', 'last_name': 'None', 'birth_date': '1909-08-10', 'death_date': '1961-02-26', 'place_of_birth': 'Fez', 'wikipedia_url': 'https://ar.wikipedia.org/wiki/%D9%85%D8%AD%D9%85%D8%AF_%D8%A7%D9%84%D8%AE%D8%A7%D9%85%D8%B3_%D8%A8%D9%86_%D9%8A%D9%88%

It is finally time to create a `get_leaders()` function for the above code. You will build on it later-on. This function takes no parameter. Inside it, you will need to:
1. define the urls
2. get the cookies
2. get the countries
3. loop over them and save their leaders in a dictionary
4. return the dictionary

In [28]:
# < 15 lines
def get_leaders():

    countries = "https://country-leaders.onrender.com/countries"
    cookie_url = "https://country-leaders.onrender.com/cookie"
    
    req = requests.get(cookie_url)
    cookies = req.cookies
    countries = requests.get("https://country-leaders.onrender.com/countries", cookies=cookies).json()


    leaders_per_country = {}
    for country in countries:
        leaders= requests.get("https://country-leaders.onrender.com/leaders", params={"country": country},cookies=cookies).json()
        leaders_per_country[country] = leaders
    return leaders_per_country
    

    
    

Test your function, save the result in the `leaders_per_country` dictionary and check its ouput.

In [29]:
# 2 lines
leader = get_leaders()
print(leader)


{'ma': [{'id': 'Q57553', 'first_name': 'Mohammed', 'last_name': 'None', 'birth_date': '1963-08-21', 'death_date': None, 'place_of_birth': 'Rabat', 'wikipedia_url': 'https://ar.wikipedia.org/wiki/%D9%85%D8%AD%D9%85%D8%AF_%D8%A7%D9%84%D8%B3%D8%A7%D8%AF%D8%B3_%D8%A8%D9%86_%D8%A7%D9%84%D8%AD%D8%B3%D9%86', 'start_mandate': '1999-07-23', 'end_mandate': None}, {'id': 'Q69103', 'first_name': 'Hassan', 'last_name': 'None', 'birth_date': '1929-07-09', 'death_date': '1999-07-20', 'place_of_birth': 'Rabat', 'wikipedia_url': 'https://ar.wikipedia.org/wiki/%D8%A7%D9%84%D8%AD%D8%B3%D9%86_%D8%A7%D9%84%D8%AB%D8%A7%D9%86%D9%8A_%D8%A8%D9%86_%D9%85%D8%AD%D9%85%D8%AF', 'start_mandate': '1961-02-26', 'end_mandate': '1999-07-23'}, {'id': 'Q193874', 'first_name': 'Mohammed', 'last_name': 'None', 'birth_date': '1909-08-10', 'death_date': '1961-02-26', 'place_of_birth': 'Fez', 'wikipedia_url': 'https://ar.wikipedia.org/wiki/%D9%85%D8%AD%D9%85%D8%AF_%D8%A7%D9%84%D8%AE%D8%A7%D9%85%D8%B3_%D8%A8%D9%86_%D9%8A%D9%88%

## Extracting data from Wikipedia

Query one of the leaders' Wikipedia urls and display its `text` (not JSON).

In [30]:
# 3 lines
req = requests.get("https://nl.wikipedia.org/wiki/Guy_Verhofstadt")
leader = req.text
print(leader)


<!DOCTYPE html>
<html class="client-nojs vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-sticky-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-menu-pinned-disabled vector-feature-limited-width-clientpref-1 vector-feature-limited-width-content-enabled vector-feature-custom-font-size-clientpref-1 vector-feature-appearance-enabled vector-feature-appearance-pinned-clientpref-1 vector-feature-night-mode-disabled skin-theme-clientpref-day vector-toc-available" lang="nl" dir="ltr">
<head>
<meta charset="UTF-8">
<title>Guy Verhofstadt - Wikipedia</title>
<script>(function(){var className="client-js vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-sticky-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-menu-pinned-disabled vector-feature-

Ouch! You get the raw HTML code of the webpage. If you try to deal with it without tools, you will be there all night. Instead, use the [beautiful soup 4](https://www.crummy.com/software/BeautifulSoup/bs4/doc/) *external* library. You will find more info about it [here](../../2.python/2.python_advanced/05.Scraping/1.beautifulsoup_basic.ipynb) and [here](../../2.python/2.python_advanced/05.Scraping/2.beautifulsoup_advanced.ipynb)

Using the Quickstart section, start by importing the library and loading the output of your `get_text()` function.

Use the `prettify()` function and print it to take a look. You will start the actual parsing in the next step.

In [31]:
# 3 lines
from bs4 import BeautifulSoup
soup = BeautifulSoup(req.content, "html")
print(soup.prettify())


<!DOCTYPE html>
<html class="client-nojs vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-sticky-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-menu-pinned-disabled vector-feature-limited-width-clientpref-1 vector-feature-limited-width-content-enabled vector-feature-custom-font-size-clientpref-1 vector-feature-appearance-enabled vector-feature-appearance-pinned-clientpref-1 vector-feature-night-mode-disabled skin-theme-clientpref-day vector-toc-available" dir="ltr" lang="nl">
 <head>
  <meta charset="utf-8"/>
  <title>
   Guy Verhofstadt - Wikipedia
  </title>
  <script>
   (function(){var className="client-js vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-sticky-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-menu-pinned-disab

In [32]:
soup = BeautifulSoup(req.content, 'html')

text_content = soup.get_text()
print("Text:" ,text_content)

Text: 


Guy Verhofstadt - Wikipedia



































Naar inhoud springen







Hoofdmenu





Hoofdmenu
naar zijbalk verplaatsen
verbergen



		Navigatie
	


HoofdpaginaVind een artikelVandaagEtalageCategorieënRecente wijzigingenNieuwe artikelenWillekeurige pagina





		Informatie
	


GebruikersportaalSnelcursusHulp en contactDoneren



















Zoeken











Zoeken






















Uiterlijk
















Account aanmaken

Aanmelden








Persoonlijke hulpmiddelen





 Account aanmaken Aanmelden





		Pagina's voor uitgelogde redacteuren meer lezen



BijdragenOverleg




























Inhoud
naar zijbalk verplaatsen
verbergen




Top





1
Politieke carrière




Toggle Politieke carrière subsection





1.1
Het begin








1.2
"Baby Thatcher" en "Da Joenk"








1.3
De Burgermanifesten








1.4
Verhofstadt I – een maatschappelijk project








1.5
Verhofstadt II – een sociaal project








1.6
2004 – annus horribilis v

That looks better but you need to extract the right part of the webpage: the text of the first paragraph.

It is a bit tricky because Wikipedia pages slightly differ in structure from one language to the next. We cannot simply get the text for the first HTML paragraph.

You will start by getting all the HTML paragraphs from the HTML source and saving them in the `paragraphs` variable.

Use the documentation or google the appropriate keywords.

In [33]:
paragraphs = soup.find("p")
print(paragraphs.text)

Guy Maurice Marie-Louise Verhofstadt[5] (uitspraakⓘ) (Dendermonde, 11 april 1953) is een Belgisch politicus voor de Open Vlaamse Liberalen en Democraten (Open Vld). Hij was premier van België van 12 juli 1999 tot 20 maart 2008 in drie regeringen. Hij beëindigde zijn actieve politieke carrière in het Europees Parlement, waar hij van 2009 tot 2019 fractieleider van de Alliantie van Liberalen en Democraten voor Europa (ALDE) was.[6]



In [34]:
# 2 lines
paragraphs= []
for tag in soup.find_all("p"):
    paragraphs.append(tag)
print(paragraphs[0].text)

Guy Maurice Marie-Louise Verhofstadt[5] (uitspraakⓘ) (Dendermonde, 11 april 1953) is een Belgisch politicus voor de Open Vlaamse Liberalen en Democraten (Open Vld). Hij was premier van België van 12 juli 1999 tot 20 maart 2008 in drie regeringen. Hij beëindigde zijn actieve politieke carrière in het Europees Parlement, waar hij van 2009 tot 2019 fractieleider van de Alliantie van Liberalen en Democraten voor Europa (ALDE) was.[6]



In [35]:
# 2 lines
for tag in soup.find_all("p", limit=1):
    print(tag.text)

Guy Maurice Marie-Louise Verhofstadt[5] (uitspraakⓘ) (Dendermonde, 11 april 1953) is een Belgisch politicus voor de Open Vlaamse Liberalen en Democraten (Open Vld). Hij was premier van België van 12 juli 1999 tot 20 maart 2008 in drie regeringen. Hij beëindigde zijn actieve politieke carrière in het Europees Parlement, waar hij van 2009 tot 2019 fractieleider van de Alliantie van Liberalen en Democraten voor Europa (ALDE) was.[6]



If you try different urls, you might find that the paragraph you want may be at a different index each time.

That is where you need to be clever and ask yourself what would be a reliable way to identify the right index ie. which string matches only the first paragraph whatever the language...

Spend a good 30 minutes on the problem and brainstorm with your fellow learners. If you come out empty handed, ask your coach.

1. Loop over the HTML paragraphs
2. When you have identified the correct one
  * store the [text](https://www.crummy.com/software/BeautifulSoup/bs4/doc/#output) inside the `first_paragraph` variable
  * exit the loop

In [36]:
# <10 lines
rq= requests.get("https://fr.wikipedia.org/wiki/Emmanuel_Macron")
soup1 = BeautifulSoup(rq.text, "html")

first_paragraphs= []
for tag in soup1.find_all("p"):
    if tag.find('b'):
        first_paragraphs.append(tag.text)
print(first_paragraphs)




["Emmanuel Macron ([ɛmanɥɛl makʁɔ̃][e] Écouter), né le 21 décembre 1977 à Amiens (Somme), est un haut fonctionnaire, banquier d'affaires et homme d'État français. Il est président de la République française depuis le 14 mai 2017.\n"]


At this stage, you can create a function to maintain consistency in your code. We will give you its *skeleton*, you will copy the code you wrote and make it work inside a function.

Don't forget to test your function.

In [37]:

def get_first_paragraph(wikipedia_url):
    print(wikipedia_url)
    rq= requests.get(wikipedia_url)
    soup = BeautifulSoup(rq.text, "html")

    for tag in soup.find_all("p"):
        if tag.find('b'):
            return tag.text

test = get_first_paragraph("https://fr.wikipedia.org/wiki/Elio_Di_Rupo")
print(test)

https://fr.wikipedia.org/wiki/Elio_Di_Rupo
Elio Di Rupo, né le 18 juillet 1951 à Morlanwelz-Mariemont, est un homme d'État belge francophone, président du Parti socialiste (PS) de 1999 à 2019, Premier ministre de Belgique de 2011 à 2014 et trois fois ministre-président de Wallonie[1],[2] de 1999 à 2000, de 2005 à 2007 et depuis le 13 septembre 2019.



In [38]:
# 10 lines
# def get_first_paragraph(wikipedia_url):
#   print(wikipedia_url) # keep this for the rest of the notebook
#   [insert your code]
#   return first_paragraph

def get_first_paragraph(wikipedia_url):
    print(wikipedia_url)
    rq= requests.get(wikipedia_url)
    soup = BeautifulSoup(rq.text, "html")

    for tag in soup.find_all("p"):
        if tag.find('b'):
            first_paragraph= tag.text
    return first_paragraph

test2 = get_first_paragraph("https://fr.wikipedia.org/wiki/Elio_Di_Rupo")
print(test2)


https://fr.wikipedia.org/wiki/Elio_Di_Rupo
Elio Di Rupo, né le 18 juillet 1951 à Morlanwelz-Mariemont, est un homme d'État belge francophone, président du Parti socialiste (PS) de 1999 à 2019, Premier ministre de Belgique de 2011 à 2014 et trois fois ministre-président de Wallonie[1],[2] de 1999 à 2000, de 2005 à 2007 et depuis le 13 septembre 2019.



In [39]:
# Test: 3 lines
test = get_first_paragraph("https://en.wikipedia.org/wiki/Joe_Biden")
print(test)

https://en.wikipedia.org/wiki/Joe_Biden
Joseph Robinette Biden Jr. (/ˈbaɪdən/ ⓘ BY-dən; born November 20, 1942) is an American politician who is the 46th and current president of the United States since 2021. A member of the Democratic Party, he previously served as the 47th vice president from 2009 to 2017 under President Barack Obama and represented Delaware in the United States Senate from 1973 to 2009.



### Regular expressions to the rescue

Now that you have extracted the content of the first paragraph, the only thing that remains to finish your Wikipedia scraper is to sanitize the output.

Indeed some Wikipedia references, HTML code, phonetic pronunciation etc. may linger. You might find *regular expressions* handy to get rid of them and obtain pristine text. You will find some useful documentation about regular expressions [here](../../2.python/2.python_advanced/03.Regex/regex.ipynb)

Once you have one of your regex working online, try it in the cell below. 

Hints: 
* Check the `sub()` method documentation.
* Make sure to test urls in different languages. Some may look good but other do not.

In [40]:
# 3 lines
import re
string = test
test3= re.sub("/.*;","", string)
print(test3)



Joseph Robinette Biden Jr. ( born November 20, 1942) is an American politician who is the 46th and current president of the United States since 2021. A member of the Democratic Party, he previously served as the 47th vice president from 2009 to 2017 under President Barack Obama and represented Delaware in the United States Senate from 1973 to 2009.



Overwrite the `get_first_paragraph()` function by applying your regex to the first paragraph before returning it.

In [41]:
# 10 lines
def get_first_paragraph(wikipedia_url):
    print(wikipedia_url)
    rq= requests.get(wikipedia_url)
    soup = BeautifulSoup(rq.text, "html")

    for tag in soup.find_all("p"):
        if tag.find('b'):
            first_paragraph= tag.text
            first_paragraph= re.sub("/.*;","", first_paragraph)
    return first_paragraph

test2 = get_first_paragraph("https://en.wikipedia.org/wiki/Newt_Gingrich")
print(test2)

https://en.wikipedia.org/wiki/Newt_Gingrich
Newton Leroy Gingrich ( born June 17, 1943) is an American politician and author who served as the 50th speaker of the United States House of Representatives from 1995 to 1999. A member of the Republican Party, he was the U.S. representative for Georgia's 6th congressional district serving north Atlanta and nearby areas from 1979 until his resignation in 1999. In 2012, Gingrich unsuccessfully ran for the Republican nomination for president of the United States.



Come up with other regexes to capture other patterns and sanitize the outputs completely. Modify your `get_first_paragraph()` function accordingly.

In [42]:
def get_first_paragraph(wikipedia_url):
    print(wikipedia_url)

    wikipedia_url= re.sub(r"^https://[a-z]{2}\.", "https://en.", wikipedia_url)

    rq= requests.get(wikipedia_url)
    soup = BeautifulSoup(rq.text, "html")

    for tag in soup.find_all("p"):
        if tag.find('b'):
            first_paragraph= tag.text
            first_paragraph= re.sub("/.*;","", first_paragraph)
            first_paragraph = re.sub(r" \(.*; ","(",first_paragraph)
                     
    return first_paragraph


test2 = get_first_paragraph("https://pl.wikipedia.org/wiki/Donald_Tusk")
print(test2)


https://pl.wikipedia.org/wiki/Donald_Tusk
https://en.wikipedia.org/wiki/Donald_Tusk


Donald Franciszek Tusk ( born 22 April 1957) is a Polish politician and historian who has been Prime Minister of Poland since 2023, having previously held the post from 2007 to 2014. From 2014 to 2019 Tusk was President of the European Council, and from 2019 to 2022 he was the president of the European People's Party (EPP). He co-founded the Civic Platform (PO) party in 2001 and has twice been its leader, first from 2003 to 2014 and again since 2021.



## Putting it all together

Let's go back to your `get_leaders()` function and update it with an *inner* loop over each leader. You will query the url provided and extract the first paragraph using the `get_first_paragraph()` function you just finished. You will then update that `leader`'s dictionary and move on to the next one.

Notice, the rest of the code should not change since you modify the leader's data one by one.

In [43]:
def get_leaders():

    countries_url = "https://country-leaders.onrender.com/countries"
    cookie_url = "https://country-leaders.onrender.com/cookie"
    leaders_url = "https://country-leaders.onrender.com/leaders"
    
    req = requests.get(cookie_url)
    cookies = req.cookies
    countries = requests.get(countries_url, cookies=cookies).json()


    leaders_per_country = {}
    for country in countries:
        leaders_list= requests.get(leaders_url, params={"country": country},cookies=cookies).json()
        leaders = leaders_list
        leaders_per_country[country] = leaders
        for leader in leaders:
            url = leader.get("wikipedia_url")
            para= get_first_paragraph(url)
            leader["first_paragraph"]= para
            print(leader)
            

            
    return leaders_per_country
    

In [44]:
# Check the output of your function (2 lines)
leaders = get_leaders()
print(leaders)

https://ar.wikipedia.org/wiki/%D9%85%D8%AD%D9%85%D8%AF_%D8%A7%D9%84%D8%B3%D8%A7%D8%AF%D8%B3_%D8%A8%D9%86_%D8%A7%D9%84%D8%AD%D8%B3%D9%86
https://en.wikipedia.org/wiki/%D9%85%D8%AD%D9%85%D8%AF_%D8%A7%D9%84%D8%B3%D8%A7%D8%AF%D8%B3_%D8%A8%D9%86_%D8%A7%D9%84%D8%AD%D8%B3%D9%86
{'id': 'Q57553', 'first_name': 'Mohammed', 'last_name': 'None', 'birth_date': '1963-08-21', 'death_date': None, 'place_of_birth': 'Rabat', 'wikipedia_url': 'https://ar.wikipedia.org/wiki/%D9%85%D8%AD%D9%85%D8%AF_%D8%A7%D9%84%D8%B3%D8%A7%D8%AF%D8%B3_%D8%A8%D9%86_%D8%A7%D9%84%D8%AD%D8%B3%D9%86', 'start_mandate': '1999-07-23', 'end_mandate': None, 'first_paragraph': 'Other reasons this message may be displayed:\n'}
https://ar.wikipedia.org/wiki/%D8%A7%D9%84%D8%AD%D8%B3%D9%86_%D8%A7%D9%84%D8%AB%D8%A7%D9%86%D9%8A_%D8%A8%D9%86_%D9%85%D8%AD%D9%85%D8%AF
https://en.wikipedia.org/wiki/%D8%A7%D9%84%D8%AD%D8%B3%D9%86_%D8%A7%D9%84%D8%AB%D8%A7%D9%86%D9%8A_%D8%A8%D9%86_%D9%85%D8%AD%D9%85%D8%AF
{'id': 'Q69103', 'first_name': 'Hassan',

AttributeError: 'str' object has no attribute 'get'

Does the function crash in the middle of the loop? Chances are the cookies have expired while looping over the leaders.

Modify your function with an *exception* or check if the `status_code` is a cookie error. In either case, get new cookies and query the api again.

If your code did not crash,

In [50]:
# < 25 lines

def get_leaders():

    countries_url = "https://country-leaders.onrender.com/countries"
    cookie_url = "https://country-leaders.onrender.com/cookie"
    leaders_url = "https://country-leaders.onrender.com/leaders"
    
    req = requests.get(cookie_url)
    cookies = req.cookies
    countries = requests.get(countries_url, cookies=cookies).json()


    leaders_per_country = {}
    for country in countries:
        while True:
            try:
                leaders_list = requests.get(leaders_url, params={"country": country},cookies=cookies).json()
                if countries.status_code == 403:
                    req = requests.get(cookie_url)
                    req.cookies
                else:   
                    leaders = leaders_list
                    leaders_per_country[country] = leaders
                    for leader in leaders:
                        url = leader.get("wikipedia_url")
                        para = get_first_paragraph(url)
                        leader["first_paragraph"]= para
                        print(leader)
                    break
            except Exception as e:
                print(e)
                break 

            
    return leaders_per_country

Check the output of your function again.

In [51]:
test = get_leaders()
print(test)

'list' object has no attribute 'status_code'
'list' object has no attribute 'status_code'
'list' object has no attribute 'status_code'
'list' object has no attribute 'status_code'
'list' object has no attribute 'status_code'
{}


Well done! It took a while however... Let's speed things up. The main *bottleneck* is the loop. We call on the Wikipedia website many times.

You will use the same *session* to call all the wikipedia pages. Check the *Advanced Usage* section of the Requests module's documentation.

Start by modifying the `get_first_paragraph()` function to accept a session parameter and adjust the `get()` method call.

In [ ]:
# < 20 lines


Modify your `get_leaders()` function to make use of a single session for all the Wikipedia calls.
1. create a `Session` object outside of the loop over countries.
2. pass it to the `get_first_paragraph()` function as an argument.

In [ ]:
# <25 lines


Test your new functions.



## Saving your hard work

The final step is to save the ``leaders_per_country`` dictionary in the `leaders.json` file using the [json](https://docs.python.org/3/library/json.html) module. Check out the `with` statement.

In [ ]:
# 3 lines


Make sure the file can be read back. Write the code to read the file. And check the variables are the same.

In [ ]:
# 3 lines


Make a function `save(leaders_per_country)` to call this code easily.

In [ ]:
# 3 lines


In [ ]:
# Call the function (1 line)


## Tidy things up in a stand-alone python script

Congratulations! You now have a working scraper! However, your code is scattered throughout this notebook along side the tutorials. Hardly production ready...

Copy and paste what you need in a separate `leaders_scraper.py` file.
Make sure it works by calling `python3 leaders_scraper.py`

## To go further

If you want to practice scraping, you can read this section and tackle the exercise.

You have noticed the API returns very partial results for country leaders. Many are missing. Overwrite the `get_leaders()` function to get its list from Wikipedia and extract their *personal details* from the frame on the side.

Good luck!